In [10]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

excel_file = '/content/SPORADIC INFECTIOUS DISEASES.xlsx'
sheet_name = 'Sheet1'

try:
    df = pd.read_excel(excel_file, sheet_name=sheet_name)
except FileNotFoundError:
    print(f"Error: The file '{excel_file}' was not found.")
    exit()
except Exception as e:
    print(f"An error occurred while reading the Excel file: {e}")
    exit()


required_columns = ['infectious disease', 'Record period', 'Location', 'Cumulative cases (deaths) reported during the record period', 'Cumulative cases (deaths) reported since 01/01/2023']  # Match column names in your Excel
missing_columns = [col for col in required_columns if col not in df.columns]

if missing_columns:
    print(f"Error: The following columns are missing from the Excel file: {missing_columns}")
    exit()


df.columns = ['infectious_disease', 'Record_period', 'Location', 'Cumulative_cases', 'Cumulative_cases_since_01012023']

def extract_date(record_period):
    try:
        date = pd.to_datetime(record_period, errors='raise')
        return date
    except ValueError:
        parts = record_period.split('-')
        if len(parts) > 1:
            try:
                date = pd.to_datetime(parts[0], errors='raise')
                return date
            except ValueError:
                return None
        else:
            return None
    except Exception as e:
        print(f"Error parsing date: {e} - {record_period}")
        return None


df['Record_period'] = df['Record_period'].apply(extract_date)


df.dropna(subset=['Record_period'], inplace=True)


df['Cumulative_cases'] = df['Cumulative_cases'].astype(str).str.extract('(\d+)').astype(float)
df['Cumulative_cases_since_01012023'] = df['Cumulative_cases_since_01012023'].astype(str).str.extract('(\d+)').astype(float)


df.dropna(subset=['Cumulative_cases', 'Cumulative_cases_since_01012023'], inplace=True)


location_mapping = {
    'Panama': {'lat': 8.5380, 'lon': -80.7821},
    'Colombia': {'lat': 4.5709, 'lon': -74.2973},
    'Korea': {'lat': 35.9078, 'lon': 127.7669},
    'USA': {'lat': 37.0902, 'lon': -95.7129},
    'Bangladesh': {'lat': 23.6850, 'lon': 90.3563},
    'Brazil': {'lat': -14.2350, 'lon': -51.9253},
    'Kenya': {'lat': -0.0236, 'lon': 37.9062},
    'Sri Lanka': {'lat': 7.8731, 'lon': 80.7718},
    'Afghanistan': {'lat': 33.9391, 'lon': 67.7100},
    'Pakistan': {'lat': 30.3753, 'lon': 69.3451},
    'The Republic of North Macedonia': {'lat': 41.6086, 'lon': 21.7453},
    'Kazakstan': {'lat': 48.0196, 'lon': 66.9237},
    'Kyrgyzstan': {'lat': 42.8746, 'lon': 74.5698},
    'Senegal': {'lat': 14.4974, 'lon': -14.4524},
    'Iraq': {'lat': 33.2232, 'lon': 43.6793},
    'Australia': {'lat': -25.2744, 'lon': 133.7751},
    'Philippines': {'lat': 12.8797, 'lon': 121.7740},
    'The Republic of Vanuatu': {'lat': -15.3767, 'lon': 166.9592},
    'Nigeria': {'lat': 9.0820, 'lon': 8.6753},
    'Algeria': {'lat': 28.0339, 'lon': 1.6596},
    'Germany': {'lat': 51.1657, 'lon': 10.4515},
    'Switzerland': {'lat': 46.8182, 'lon': 8.2275},
    'Bolivia': {'lat': -16.2902, 'lon': -63.5887},
    'Denmark': {'lat': 56.2639, 'lon': 9.5018},
    'Taiwan, China': {'lat': 23.6978, 'lon': 120.9605},
    'Hong Kong, China': {'lat': 22.3193, 'lon': 114.1694},
    'France': {'lat': 46.2276, 'lon': 2.2137},
    'Romania': {'lat': 45.9432, 'lon': 24.9668},
    'Republic of Serbia': {'lat': 44.0165, 'lon': 21.0059},
    'Greece': {'lat': 39.0742, 'lon': 21.8243},
    'Hungary': {'lat': 47.1625, 'lon': 19.5033},
    'Italy': {'lat': 41.8719, 'lon': 12.5674},
    'The United Arab Emirates': {'lat': 23.4241, 'lon': 53.8478}
}


df['lat'] = df['Location'].map(lambda x: location_mapping[x]['lat'] if x in location_mapping else None)
df['lon'] = df['Location'].map(lambda x: location_mapping[x]['lon'] if x in location_mapping else None)


if df['lat'].isnull().any() or df['lon'].isnull().any():
    print("Warning: Some locations are missing latitude/longitude data. These will be excluded from map-based visualizations.")
    df.dropna(subset=['lat', 'lon'], inplace=True)

df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

# 1. Time Series Analysis of Cases (Scatter Plot) - KEPT AS SCATTER
fig_time_series = px.scatter(df, x='Record_period', y='Cumulative_cases', color='infectious_disease',
                                title='Cumulative Cases Over Time')
fig_time_series.update_layout(xaxis_title='Record Period', yaxis_title='Cumulative Cases', template="plotly_white")

fig_time_series.show()

# 2. Cases by Location (Bar Chart)
fig_location_cases = px.bar(df, x='Location', y='Cumulative_cases', color='infectious_disease',
                                    title='Cumulative Cases by Location', barmode='group')
fig_location_cases.update_layout(xaxis_title='Location', yaxis_title='Cumulative Cases', template="plotly_white")

fig_location_cases.show()

# 3. Interactive Map
fig_map = px.scatter_geo(df,
                                    lat='lat',
                                    lon='lon',
                                    color='infectious_disease',
                                    hover_name='Location',
                                    size='Cumulative_cases',
                                    projection='natural earth',
                                    title='Geographic Distribution of Infectious Diseases')

fig_map.update_layout(template="plotly_white")

fig_map.show()

# 4. Disease Comparison (Subplots) - CHANGED TO LINE PLOT
diseases = df['infectious_disease'].unique()
fig_subplots = make_subplots(rows=len(diseases), cols=1, subplot_titles=diseases)

for i, disease in enumerate(diseases):
    df_disease = df[df['infectious_disease'] == disease].sort_values('Record_period')
    fig_subplots.add_trace(go.Scatter(x=df_disease['Record_period'], y=df_disease['Cumulative_cases'],
                                                        mode='lines', name=disease), row=i + 1, col=1)  # Changed to lines

fig_subplots.update_layout(height=2000, title_text="Cumulative Cases for Each Disease Over Time", template="plotly_white")
fig_subplots.update_xaxes(title_text="Record Period")
fig_subplots.update_yaxes(title_text="Cumulative Cases")

fig_subplots.show()

# 5. Interactive Table (for detailed inspection)
fig_table = go.Figure(data=[go.Table(
    header=dict(values=list(df.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df[col].to_list() for col in df.columns],
                fill_color='lavender',
                align='left'))
])
fig_table.update_layout(title_text="Detailed Data Table", template="plotly_white")

fig_table.show()

# 6. Cumulative Cases reported since 01/01/2023 (Bar Chart)
fig_cases_since = px.bar(df, x='Location', y='Cumulative_cases_since_01012023', color='infectious_disease',
                                        title='Cumulative Cases Since 01/01/2023 by Location', barmode='group')
fig_cases_since.update_layout(xaxis_title='Location', yaxis_title='Cumulative Cases Since 01/01/2023', template="plotly_white")

fig_cases_since.show()

# 7. Grouped Bar Chart of Cumulative Cases and Cases since 2023
fig = go.Figure()

# Add the first trace for Cumulative Cases
fig.add_trace(go.Bar(
    x=df['Location'],
    y=df['Cumulative_cases'],
    name='Cumulative Cases',
    marker_color='skyblue'
))

# Add the second trace for Cumulative Cases Since 01/01/2023
fig.add_trace(go.Bar(
    x=df['Location'],
    y=df['Cumulative_cases_since_01012023'],
    name='Cumulative Cases Since 01/01/2023',
    marker_color='lightcoral'
))

# Update the layout
fig.update_layout(
    title='Cumulative Cases vs. Cases Since 01/01/2023 by Location',
    xaxis_title='Location',
    yaxis_title='Number of Cases',
    barmode='group',
    template="plotly_white"
)

fig.show()

# 8. Pie Chart of Disease Distribution (Based on Sum of Cumulative Cases)
disease_cumulative_cases = df.groupby('infectious_disease')['Cumulative_cases'].sum().reset_index()
fig_pie = px.pie(disease_cumulative_cases, names='infectious_disease', values='Cumulative_cases',
                 title='Distribution of Infectious Diseases (Based on Total Cumulative Cases)')
fig_pie.update_layout(template="plotly_white")
fig_pie.show()

<ipython-input-10-c19eb2948451>:37: UserWarning:

Parsing dates in  %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.

<ipython-input-10-c19eb2948451>:31: UserWarning:

Parsing dates in  %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.

